# AMAL - TP1
## Définition de fonctions en pyTorch

### 1. Fonction dans un graphe

On utilise la règle de la dérivée en chaîne.

$$\frac{\partial (L \circ h)}{\partial x_i}(x) = \sum_{j=1}^{p} \frac{\partial(L \circ h)}{\partial\ y_j} \cdot \frac{\partial h_j}{\partial x_i}(x)=\sum_{j=1}^{p} (\nabla L)_j \cdot \frac{\partial h_j}{\partial x_i}(x)$$

La dérivée de la composition $L\circ h$ par rapport à $x_i$ dépend des dérivées partielles de $h_j$ par rapport à $x_i$ et des dérivées de la fonctions de coût $L$ par rapport à ses sorties $y_j$. 

Pour calculer le gradient d'une fonction composée, il suffit de connaître le gradient de chaque fonction par rapport à chacune de ses entrées et appliquer une dérivation chaînée. 

Dans le contexte des réseaux de neurones, chaque couche du réseau est essentiellement une fonction composée de transformations linéaires (pondérées par les poids des connexions) et de fonctions d'activation non linéaires. 

Lors de l'apprentissage, on calcule les gradients des poids par rapport à la fonction de coût, ce qui implique d'appliquer la règle de la chaîne pour obtenir ces gradients

**Forward** Calcul de $y = h(x)$ en fonction de $x$. Les donénes d'entrées sont introduites dans le réseau de neurones puis passent par chaque couche du réseau, où des transformations linéaires et des fonctions d'activation non linéaires sont appliquées. Génère les prédictions (sorties) du réseau

**Backward** Calcul de $\frac{h}{x}(x)$ et retour en prenant en compte les $\nabla L(y)$ passés en paramètres et de données dérivées de la phase forward $(x)$. Les gradients de la perte par rapport aux paramètres du modèles sont calculés. Calcul itératif, commençant par la dernière couche du modèle (sortie) et se propageant ensuite en sens inverse à travers les couches du modèle.

### 2. Application : Régression Linéaire

#### 2.1 Calcul du gradient (scalaire)

Le modèle de régression linéaire peut être vu comme la composition de 2 éléments:
- une fonction linéaire, responsable du calcul de la prédiction: $$\hat{y} = f(x,w,b) = x \cdot w + b$$
- un coût aux moindres carrés (MSE): $$mse(\hat{y}, y)=(\hat{y}-y)^2$$

Afin d'effectuer une descente de gradient, il faut calculer le gradient du coût par rapport aux paramètres w et b. Nous allons calculer ce gradient grâce au chaînage des dérivées partielles. 

Nous supposons que le coût est défini de la manière suivante: $$C(x,w,b,y) = mse(f(x,w,b),y)$$ où $x$ est l'entrée, $y$ la sortie désirée dans $\mathbb{R}$ et $w \in \mathbb{R}^n$ et $b \in \mathbb{R}$


##### Fonction Loss MSE

Pour la fonction de coût MSE, les dérivées par rapport à $\hat{y}$ et $y$ sont les mêmes.

Calcul de $\frac{\partial(L \circ mse)}{\partial y}(\hat{y}, y)$ :

En utilisant la règle de la chaîne, nous avons $$\frac{\partial(L \circ mse)}{\partial y} = \frac{\partial mse(\hat{y}, y)}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial y}$$

$$\frac{\partial mse(\hat{y}, y)}{\partial \hat{y}} = 2(\hat{y} - y)$$

$$\frac{\partial \hat{y}}{\partial y} = 1$$

Donc, $$\frac{\partial(L \circ mse)}{\partial y} = 2(\hat{y} - y)$$ et $$\frac{\partial(L \circ mse)}{\partial \hat{y}} = 2(\hat{y} - y)$$

##### Fonction Linéaire

Pour calculer $$\frac{\partial(L \circ f)}{\partial X}(x, w, b) =  \frac{\partial(L \circ f)}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial X}$$
La première partie $\frac{\partial(L \circ f)}{\partial \hat{y}}$ est la dérivée de la perte MSE par rapport à la sortie de la couche linéaire.

La deuxième partie $\frac{\partial \hat{y}}{\partial X}$ est la dérivée de la sortie de la couche linéaire par rapport à X. 

D'où $$\frac{\partial(L \circ f)}{\partial X}(x, w, b) =  (\hat{y} - y) \cdot W$$

 Pour calculer $$\frac{\partial(L \circ f)}{\partial W}(X, W, b) =  \frac{\partial(L \circ f)}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial W}$$
 D'où $$\frac{\partial(L \circ f)}{\partial W}(X, W, b) =  (\hat{y} - y) \cdot X$$

 Pour calculer $$\frac{\partial(L \circ f)}{\partial b}(X, W, b) =  \frac{\partial(L \circ f)}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial b}$$
 D'où $$\frac{\partial(L \circ f)}{\partial b}(X, W, b) =  (\hat{y} - y) \cdot 1$$

#### 2.2 Calcul du gradient (matriciel)

On considère maintenant un cas plus général où nous travaillons avec un lot( batch) de *q* exemples (au lieu d'un) et *p* sorties (au lieu d'une).
- Nous avons toujours une fonction linéaire, responsable du calcul de la prédiction: $$ \hat{Y} = f(X, W, b) = XW + \begin{bmatrix}
b \\
\vdots \\
b
\end{bmatrix} $$
où $X \in \mathbb{R}^{q \times n}$ sont les entrées, $W \in \mathbb{R}^{n \times p}$ est la matrice de poids, et $b \in \mathbb{R}^{1 \times p}$ est le biais. La sortie $Y$ est donc une matrice dans $\mathbb{R}^{q \times p}
- un coût aux moindres carrés (MSE) que l'on généralise a un lot d'exemples et plusieurs sorties : $mse(\hat{Y}, Y) = \frac{1}{q}||\hat{Y} - Y||^2$. Notez que l'on divise par le nombre d'exemple, ceq ui permet d'avoir une magnitude du gradient qui ne vaie pas en fonction du nombre d'exemples. 

Le coût $C$ est alors défini comme $$ C(X, W, b, Y) = mse(f(X, W, b), Y)$$

##### Fonction Loss MSE

Pour calculer $$\frac{\partial L \circ mse}{\partial Y_{ij}}(\hat{Y}, Y) = \frac{\partial L \circ mse}{\partial \hat{Y}_{ij}}(\hat{Y}, Y)$$ 

Avec la méthode de dérivation en chaîne, obtient $$\frac{2}{q} \sum_{j=1}^{q}(\hat{Y} - Y)$$

NB : Pour la loss MSE les dérivées partielles sont égales.

##### Fonction Linéaire

Pour calculer $$\frac{\partial(L \circ mse)}{\partial X_{ij}}(X, W, b) =  \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) \cdot \frac{\partial f_{ij}}{\partial X_{ij}}(X, W, b)$$
D'où $$\frac{\partial(L \circ mse)}{\partial X_{ij}}(X, W, b) =  \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) \cdot W_{ij}$$


Pour calculer $$\frac{\partial(L \circ mse)}{\partial W_{ij}}(X, W, b) =  \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) \cdot \frac{\partial f_{ij}}{\partial W_{ij}}(X, W, b)$$
D'où $$\frac{\partial(L \circ mse)}{\partial W_{ij}}(X, W, b) =  \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) \cdot X_{ij}$$


Pour calculer $$\frac{\partial(L \circ mse)}{\partial b_j}(X, W, b) =  \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) \cdot \frac{\partial f_{ij}}{\partial W_{ij}}(X, W, b_j)$$
D'où $$\frac{\partial(L \circ mse)}{\partial b{j}}(X, W, b) = \frac{2}{q}\sum_{j=1}^{q}(\hat{Y} - Y)$$


##### Forme matricielle

Supposons que :
- $X$ est une matrice de forme (p, q) où p est le nombre de caractéristiques d'entrée et q est le nombre d'exemples.
- $W$ est une matrice de forme (p, r) où r est le nombre de neurones dans la couche de sortie.
- $b$ est un vecteur de forme (r, 1) où r est le nombre de neurones dans la couche de sortie.
- $\hat{Y}$ est la prédiction du modèle de forme (r, q).
$Y$ est la vérité de terrain de forme (r, q).

$$\frac{\partial L \circ mse}{\partial Y_{ij}}(\hat{Y}, Y) = \frac{\partial L \circ mse}{\partial \hat{Y}_{ij}}(\hat{Y}, Y) = \frac{2}{q}(\hat{Y} - Y)$$

$$\frac{\partial(L \circ mse)}{\partial X} = \frac{2}{q} (\hat{Y} - Y) \cdot W^T$$

$$\frac{\partial(L \circ mse)}{\partial W} = \frac{2}{q} X \cdot (\hat{Y} - Y)^T $$

$$\frac{\partial(L \circ mse)}{\partial b} =  \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) $$

#### 2.3 Calcul du gradient de C

Maintenant que nous avons calculé toutes les dérivées partielles, il faut montrer comment la dérivée de C par rapport à chacune de ses entrées.


Dérivée par rapport à W: $$\frac{\partial C}{\partial W} = \frac{\partial mse}{\partial \hat{Y_ij}} \cdot \frac{\partial f_{ij}}{\partial W_{ij}} = \frac{2}{q}(\hat{Y_{ij}} - Y_{ij}) \cdot X_{ij}$$

Dérivée par rapport à b : $$\frac{\partial C}{\partial b} = \frac{\partial mse}{\partial \hat{Y_{ij}}} \cdot \frac{\partial f_{ij}}{\partial b_j} = \frac{2}{q} \sum_{i=1}^{q} (\hat{Y}_i - Y_i)$$

Les dérivées de la fonction coût sont essentielles pour la mise à jour des poids et du biais.